In [33]:
@everywhere using CUDAnative, CUDAdrv

In [32]:
@everywhere using CUDAnative

In [30]:
nprocs()

2

In [27]:
rmprocs(workers())

Task (done) @0x00007f24f73b5cf0

In [29]:
addprocs([("root@10.4.1.4:6666",1)], tunnel=true)

1-element Array{Int64,1}:
 6

In [34]:
@everywhere function kernel_dist(X::AbstractVector{Float32}, Y::AbstractVector{Float32}, out::AbstractVector{Float32})
    #Thread index
    #Need to do the n-1 dance, since CUDA expects 0 and Julia does 1-indexing
    i = (blockIdx().x-1) * blockDim().x + threadIdx().x
    out[i] = (X[i]-0.5)^2 + (Y[i]-0.5)^2
    #Return nothing, since we’re writing directly to the out array allocated on GPU
    return nothing
end

In [45]:
@everywhere function distmontegpu()
    samples = 50000000
    a = rand(Float32, (samples))
    b = rand(Float32, (samples));
    a_cu = CuArray(a)
    b_cu = CuArray(b)
    c_cu = similar(a_cu);
    n = length(a)

    ctx = CuCurrentContext()
    dev = device(ctx)
    max_threads = attribute(dev, CUDAdrv.MAX_THREADS_PER_BLOCK)
    threads = min(max_threads, n)
    blocks = ceil(Int, n/threads)
    
    @cuda (blocks, threads) kernel_dist(a_cu, b_cu, c_cu)
    c = Array(c_cu)
    destroy!(ctx)
    c = Array(c_cu)
    pi = 4*count(x->x<0.25,c)/length(c)
    return pi
end

In [44]:
@everywhere distmontegpu()